# Importation des librairies

In [90]:
import spacy
from spacy import displacy
import fr_core_news_lg
from prettytable import PrettyTable
from spacy.scorer import Scorer
from spacy.gold import GoldParse
from spacy.matcher import Matcher #Adding custom entities
from spacy.tokens import Span #Adding custom entities
from spacy.matcher import PhraseMatcher
import re
from flair.data import Sentence
from flair.models import SequenceTagger,  MultiTagger

In [91]:
nlp = fr_core_news_lg.load()
doc = nlp("Emma Louise s'est installée au 26 rue Alexandre 75005, Paris, France, elle est née en 2007 et elle est joignable sur 06660006.")

## a) Pos tagging

In [92]:
x = PrettyTable()
y=PrettyTable()

In [93]:
x.field_names =  ["text", "lemma_", "pos_","tag_","dep_","shape_", "alpha", ".is_stop"]

In [94]:
for token in doc:
    x.add_row([token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop])

In [95]:
options = {"distance": 130}
displacy.render(doc, style="dep", options=options) # (1)

In [96]:
print (x)

+-----------+-----------+-------+------------------------------------------------------------+------------+--------+-------+----------+
|    text   |   lemma_  |  pos_ |                            tag_                            |    dep_    | shape_ | alpha | .is_stop |
+-----------+-----------+-------+------------------------------------------------------------+------------+--------+-------+----------+
|    Emma   |    Emma   | PROPN |               PROPN__Gender=Fem|Number=Sing                |   nsubj    |  Xxxx  |  True |  False   |
|   Louise  |   Louise  | PROPN |                           PROPN                            | flat:name  | Xxxxx  |  True |  False   |
|     s'    |     se    |  PRON |                 PRON__Person=3|Reflex=Yes                  | expl:comp  |   x'   | False |   True   |
|    est    |    être   |  AUX  | AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin | aux:tense  |  xxx   |  True |   True   |
| installée | installer |  VERB |   VERB__Gender

## b) Named Entities

In [97]:
y.field_names = ["text", "start_char", "end_char", "label_"]

In [98]:
for ent in doc.ents:
    y.add_row([ent.text, ent.start_char, ent.end_char, ent.label_])   

In [99]:
displacy.render(doc, style="ent") # (2)

In [100]:
print(y)

+-------------+------------+----------+--------+
|     text    | start_char | end_char | label_ |
+-------------+------------+----------+--------+
| Emma Louise |     0      |    11    |  PER   |
|    Paris    |     55     |    60    |  LOC   |
|    France   |     62     |    68    |  LOC   |
+-------------+------------+----------+--------+


## c) Métriques d'évaluation

In [101]:
def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores# example run

In [102]:
examples = [
    ("Je suis Marie Antoinette. J'habite à London. Diplômée en 2018 et joignable sur 06660006",[(8,24,"PER"),(37,43,"LOC")]),
    ("François travaille chez la société française Zenika depuis 2011. Il était recruté initialement à Montréal.",[(0,8,"PER"),(35,44,"MISC"),(45,51,"ORG"),(107,115,"LOC")]),
    ("Frédéric, 34 ans, installé à Lyon.",[(0,8,"PER"),(29,33,"LOC")]),
    ("Amélie habite à Palaiseau depuis son arrivée en France ",[(0,6,"PER"),(16,25,"LOC"),(48,54,"LOC")]),
    ("Julien travaille à Asnières-sur-Seine chez FBI.",[(0,6,"PER"),(19,37,"LOC"),(43,46,"ORG")]),
    ("Jack est consultant chez Google.",[(0,4,"PER"),(25,31,"ORG")]),
    ("Les anglais sont chaleureux.",[(4,11,"MISC")]),
]

In [103]:
results = evaluate(nlp, examples)

In [104]:
print("Précision des entités: ",results.get("ents_p"))
print("Rappel des entités: ",results.get("ents_r"))
print("F-score des entités: ",results.get("ents_f"))
print("Métriques de l'entité PER: ",results.get("ents_per_type").get("PER"))
print("Métriques de l'entité LOC: ",results.get("ents_per_type").get("LOC"))
print("Métriques de l'entité ORG: ",results.get("ents_per_type").get("ORG"))
print("Métriques de l'entité MISC: ",results.get("ents_per_type").get("MISC"))

Précision des entités:  92.85714285714286
Rappel des entités:  81.25
F-score des entités:  86.66666666666666
Métriques de l'entité PER:  {'p': 100.0, 'r': 83.33333333333334, 'f': 90.9090909090909}
Métriques de l'entité LOC:  {'p': 83.33333333333334, 'r': 100.0, 'f': 90.9090909090909}
Métriques de l'entité ORG:  {'p': 100.0, 'r': 100.0, 'f': 100.0}
Métriques de l'entité MISC:  {'p': 0.0, 'r': 0.0, 'f': 0.0}


## Filtre des POS de Flair

In [105]:
tagger = MultiTagger.load(['pos-multi', 'fr-ner'])

2020-12-15 14:11:47,909 loading file /Users/amalbedoui/.flair/models/pos-multi-v0.1.pt
2020-12-15 14:11:50,873 loading file /Users/amalbedoui/.flair/models/fr-ner-wikiner-0.4.pt


In [106]:
phrase ="Passionnée par le développement, Emma Louise s'est installée à Paris,  Son adresse  mail est emma.louise@google.com, son compte linkedin est https://www.linkedin.com/in/el-424781150/ et elle est joignable sur +33987609876 . elle a 26 ans vu qu'elle est née le 16/11/1994 , elle est mariée et a 2 enfants. Son numéro de sécurité sociale 2 94 11 99 205 148 . Son sexe est Féminin (F) .Elle a été diplômée en 11/11/2018, à l'issue d'une formation de 5 ans d'ingénierie de 2013 à 2018 à l'Ecole Polytechnique à Palaiseau."
# text
sentence = Sentence(phrase)

# predict PoS tags
tagger.predict(sentence)

In [107]:
print(sentence.to_tagged_string())

Passionnée <VERB> par <ADP> le <DET> développement <NOUN> , <PUNCT> Emma <PROPN/B-PER> Louise <PROPN/E-PER> s'est <AUX> installée <VERB> à <ADP> Paris <PROPN/S-LOC> , <PUNCT> Son <DET> adresse <NOUN> mail <NOUN> est <AUX> emma.louise <PROPN> @ <PUNCT> google.com <SYM> , <PUNCT> son <DET> compte <NOUN> linkedin <NOUN> est <AUX> https <ADJ> :// <PUNCT> www.linkedin.com <SYM> / <PUNCT> in <X> / <PUNCT> el-424781150 <X/S-MISC> / <PUNCT> et <CCONJ> elle <PRON> est <AUX> joignable <VERB> sur <ADP> + <SYM> 33987609876 <NUM> . <PUNCT> elle <PRON> a <VERB> 26 <NUM> ans <NOUN> vu <ADV> qu'elle <PRON> est <AUX> née <VERB> le <DET> 16 <NUM> / <PUNCT> 11 <NUM> / <PUNCT> 1994 <NUM> , <PUNCT> elle <PRON> est <AUX> mariée <VERB> et <CCONJ> a <VERB> 2 <NUM> enfants <NOUN> . <PUNCT> Son <DET> numéro <NOUN> de <ADP> sécurité <NOUN> sociale <ADJ> 2 <NUM> 94 <NUM> 11 <NUM> 99 <NUM> 205 <NUM> 148 <NUM> . <PUNCT> Son <DET> sexe <NOUN> est <AUX> Féminin <PROPN/S-MISC> ( <PUNCT> F <PROPN/S-LOC> ) <PUNCT> . <PU

In [108]:
for entity in sentence.get_spans('pos-multi'):
    print(entity)

Span [1]: "Passionnée"   [− Labels: VERB (0.9949)]
Span [2]: "par"   [− Labels: ADP (1.0)]
Span [3]: "le"   [− Labels: DET (1.0)]
Span [4]: "développement"   [− Labels: NOUN (0.9999)]
Span [5]: ","   [− Labels: PUNCT (1.0)]
Span [6]: "Emma"   [− Labels: PROPN (0.9999)]
Span [7]: "Louise"   [− Labels: PROPN (0.9999)]
Span [8]: "s'est"   [− Labels: AUX (0.9983)]
Span [9]: "installée"   [− Labels: VERB (0.9985)]
Span [10]: "à"   [− Labels: ADP (1.0)]
Span [11]: "Paris"   [− Labels: PROPN (1.0)]
Span [12]: ","   [− Labels: PUNCT (1.0)]
Span [13]: "Son"   [− Labels: DET (0.999)]
Span [14]: "adresse"   [− Labels: NOUN (0.9985)]
Span [15]: "mail"   [− Labels: NOUN (0.9567)]
Span [16]: "est"   [− Labels: AUX (0.9966)]
Span [17]: "emma.louise"   [− Labels: PROPN (0.4544)]
Span [18]: "@"   [− Labels: PUNCT (0.9971)]
Span [19]: "google.com"   [− Labels: SYM (0.7381)]
Span [20]: ","   [− Labels: PUNCT (1.0)]
Span [21]: "son"   [− Labels: DET (1.0)]
Span [22]: "compte"   [− Labels: NOUN (0.9981)]
S

In [109]:
def add_inclusive(str1,str2):
    if str1.endswith(str2):
        return str1[:-len(str2)]+"("+str2+")"
    else :
        return str1+"("+str2+")"
   
def add_inclusive_adj(str1,str2,str3):
    if str1.endswith(str2):
        return str1+"("+str3+")"
    elif str1.endswith(str3):
        return str1[:-len(str3)]+str2+"("+str3+")"    

In [110]:
print(sentence.get_token)

<bound method Sentence.get_token of Sentence: "Passionnée par le développement , Emma Louise s'est installée à Paris , Son adresse mail est emma.louise @ google.com , son compte linkedin est https :// www.linkedin.com / in / el-424781150 / et elle est joignable sur + 33987609876 . elle a 26 ans vu qu'elle est née le 16 / 11 / 1994 , elle est mariée et a 2 enfants . Son numéro de sécurité sociale 2 94 11 99 205 148 . Son sexe est Féminin ( F ) . Elle a été diplômée en 11 / 11 / 2018 , à l'issue d'une formation de 5 ans d'ingénierie de 2013 à 2018 à l'Ecole Polytechnique à Palaiseau ."   [− Tokens: 112  − Token-Labels: "Passionnée <VERB> par <ADP> le <DET> développement <NOUN> , <PUNCT> Emma <PROPN/B-PER> Louise <PROPN/E-PER> s'est <AUX> installée <VERB> à <ADP> Paris <PROPN/S-LOC> , <PUNCT> Son <DET> adresse <NOUN> mail <NOUN> est <AUX> emma.louise <PROPN> @ <PUNCT> google.com <SYM> , <PUNCT> son <DET> compte <NOUN> linkedin <NOUN> est <AUX> https <ADJ> :// <PUNCT> www.linkedin.com <SYM

In [111]:
def merging_inclusive(sentence):
    strr=""
    for entity in sentence.get_spans()  :
        
        for data in entity.labels:
            
            if re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'VERB' and len(entity.text)>2 :
                #[TO DO] Tester avec l'auxiliare qui précède le verbe: si avoir, pas de changement, sinon:
                if entity.text.endswith("é") or entity.text.endswith("e"):
                    strr= strr+add_inclusive(entity.text,"e")+" "
                elif entity.text.endswith("i") or entity.text.endswith("ie"):
                    strr= strr+add_inclusive(entity.text,"ie")+" "
                elif entity.text.endswith("is") or entity.text.endswith("ise"):
                    strr= strr+add_inclusive(entity.text,"ise")+" "
                elif entity.text.endswith("t") or entity.text.endswith("te"):
                    strr= strr+add_inclusive(entity.text,"te")+" "
                elif entity.text.endswith("us") or entity.text.endswith("use"): #inclus(se)
                    strr= strr+add_inclusive(entity.text,"use")+" "
                elif entity.text.endswith("u") or entity.text.endswith("ue"):
                    strr= strr+add_inclusive(entity.text,"ue")+" "
                else :
                    strr= strr+entity.text+" "
  
            elif re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'VERB' and len(entity.text)<2 :
                strr= strr+entity.text+" "
        
        
            elif  re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'ADJ'  :
                if entity.text.endswith("ien") or entity.text.endswith("ienne"):
                    strr= strr+" "+add_inclusive_adj(entity.text,"ien","ienne")
                elif  entity.text.endswith("if") or entity.text.endswith("ive"):
                    strr= strr+" "+add_inclusive_adj(entity.text,"if","ive")
                elif  entity.text.endswith("er") or entity.text.endswith("ère"):
                    strr= strr+" "+add_inclusive_adj(entity.text,"er","ère")
                elif  entity.text.endswith("ier") or entity.text.endswith("ière"):
                    strr= strr+" "+add_inclusive_adj(entity.text,"ier","ière")
                elif  entity.text.endswith("on") or entity.text.endswith("onne"):
                    strr= strr+" "+add_inclusive_adj(entity.text,"on","ne")
                elif  entity.text.endswith("eur") or entity.text.endswith("euse"):
                    strr= strr+" "+add_inclusive_adj(entity.text,"eur","euse")
                elif  entity.text.endswith("leur") or entity.text.endswith("leure"): #meilleur(e)
                    strr= strr+" "+add_inclusive_adj(entity.text,"eur","eure")
                elif entity.text.endswith("é") or entity.text.endswith("ée"):
                    strr= strr+add_inclusive(entity.text,"ée")+" "
                else :
                    strr= strr+entity.text+" "
        
        
            elif  re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'NOUN'  :                       
                if entity.text.endswith("teur") or entity.text.endswith("trice"):
                    strr= strr+" "+add_inclusive_adj(entity.text,"teur","trice")
                elif entity.text.endswith("ieur") or entity.text.endswith("ieure"):
                    strr= strr+" "+add_inclusive_adj(entity.text,"eur","eure")
                elif entity.text.endswith("peur") or entity.text.endswith("peuse"): #développeur(euse)
                    strr= strr+add_inclusive_adj(entity.text,"eur","euse")+" "
                else :
                    strr= strr+" "+entity.text+" "
                                    
                    
                
            elif  entity.text == 'la' or entity.text == 'le' or entity.text == 'La' or entity.text == 'Le'  :                    
                strr= strr+"le/la"+" "
                
            elif  entity.text == 'il' or entity.text == 'elle' or entity.text == 'Il' or entity.text == 'Elle'   :                    
                strr= strr+"il/elle"   +" "             
            elif  re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'ADP'  :
                strr= strr+" "+entity.text+" "
            elif  re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'AUX'  :
                strr= strr+" "+entity.text+" "
            elif  re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'DET'  :
                strr= strr+entity.text  +" "  
            elif  re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'CCONJ'  :
                strr= strr+" " +entity.text+" "    
            elif  entity.text == '.' or entity.text == ','  :
                strr= strr+" " +entity.text+" "    
                                                       
                               
            else :
                 strr= strr+""+entity.text
    return strr

In [112]:
entitiesListName = []
def merging_entities(sentence):
    entitiesList = []

    
    for entity in sentence.get_spans('fr-ner')  :
        
        for data in entity.labels:
            if re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'PER' and len(entity.text)>2 :
                #strr= strr+"[REDACTEDPER]"
                entitiesList.append(entity.text)
                entitiesListName.append('PER')
                
            if re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'LOC ' and len(entity.text)>2 :
                #strr= strr+"[REDACTEDLOC]"
                entitiesList.append(entity.text)
                entitiesListName.append('LOC')
                
            if re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'MISC' and len(entity.text)>2 :
                #strr= strr+"[REDACTEDMISC]"
                entitiesList.append(entity.text)
                entitiesListName.append('MISC')
                
                                                
            if re.sub(r'\([^)]*\)', '', str(data))[:-1] == 'ORG' and len(entity.text)>2 :
               # strr= strr+"[REDACTEDORG]"
                entitiesList.append(entity.text)
                entitiesListName.append('ORG')
                
        
    return entitiesList

In [113]:
merging_entities(sentence)

['Emma Louise', 'el-424781150', 'Féminin', 'Polytechnique']

In [114]:
i=0
for ent in merging_entities(sentence) :
    if ent in phrase:
        phrase=phrase.replace(ent,entitiesListName[i])
    i=i+1    
    
phrase    

"Passionnée par le développement, PER s'est installée à Paris,  Son adresse  mail est emma.louise@google.com, son compte linkedin est https://www.linkedin.com/in/MISC/ et elle est joignable sur +33987609876 . elle a 26 ans vu qu'elle est née le 16/11/1994 , elle est mariée et a 2 enfants. Son numéro de sécurité sociale 2 94 11 99 205 148 . Son sexe est MISC (F) .Elle a été diplômée en 11/11/2018, à l'issue d'une formation de 5 ans d'ingénierie de 2013 à 2018 à l'Ecole ORG à Palaiseau."

In [115]:
sentence2 = Sentence(phrase)
# predict PoS tags
tagger.predict(sentence2)

In [116]:
merging_inclusive(sentence2)

"Passionné(e)  par le/la  développement  , PER s'est installé(e)  à Paris , Son  adresse  mail  est emma.louise@google.com , son  compte  linkedin  est https ://www.linkedin.com/in/MISC/ et il/elle  est joignabl(e)  sur +33987609876 . il/elle a 26 ans vuqu'elle est né(e) le/la 16/11/1994 , il/elle  est marié(e)  et a 2 enfants  . Son  numéro  de  sécurité sociale 2941199205148 . Son  sexe  est MISC(F) . il/elle  a  été diplômé(e)  en 11/11/2018 ,  à l'issue d'une  formation  de 5 ans d'ingénierie  de 2013 à 2018 à l'Ecole ORG à Palaiseau . PERParisMISCMISCFORGPalaiseau"

In [117]:
entitiesListName

['PER', 'MISC', 'MISC', 'ORG', 'PER', 'MISC', 'MISC', 'ORG']

## Ajout de nouvelles entités grâce au rule-based entities de Spacy

In [118]:
matcher = Matcher(nlp.vocab)

def add_email(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="EMAIL")
    doc.ents += (entity,)
    
def add_url(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="URL")
    doc.ents += (entity,)

def add_tel(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="TEL")
    doc.ents += (entity,)
    
def add_Sdate(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="S-DATE")
    doc.ents += (entity,)
def add_date(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="DATE")
    doc.ents += (entity,)
        
def add_verb(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="VERB_FEM")
    doc.ents += (entity,)
def add_adj(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="adj fem")
    doc.ents += (entity,)
def add_ssNum(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="NUM_SEC_SO")
    doc.ents += (entity,)
def add_age(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="AGE")
    doc.ents += (entity,)
def add_situation_fam(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="SIT_FAM")
    doc.ents += (entity,)
def add_formation(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="FORMATION")
    doc.ents += (entity,)
def add_enfants(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="nbr_enfants")
    doc.ents += (entity,)
def add_sexe(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="sexe")
    doc.ents += (entity,)

def add_verb(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="verb")
    doc.ents += (entity,)

In [119]:
patternMail = [{"LIKE_EMAIL": True}]
patternUrl = [{"LIKE_URL": True}]
patternTel = [{"TEXT": {"REGEX": "^\+*\d{11}$"}}]
patternSecS = [{"TEXT": {"REGEX": "^\+*\d{13}$"}}]
#[{"SHAPE": "d"}, {"SHAPE": "dd"}, {"SHAPE": "dd"}, {"SHAPE": "dd"}, {"SHAPE": "ddd"}, {"SHAPE": "ddd"}]
patternSDate = [{"SHAPE": "dddd", "LENGTH": 4}]
patternDate = [{"TEXT": {"REGEX": "^(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](19|20)\d\d$"}}]
patternVerbFem = [{"POS": "VERB","TEXT": {"REGEX": "ée$"}}]
patternAge = [{"ORTH": "age"}, {"ORTH": ":"}, {}]
patternAge2 = [{"ORTH": "a"},{},{"ORTH": "ans"} ]
patternFormation = [{"ORTH": "de"},{},{"ORTH": "ans"} ]
patternEnfants = [{},{"ORTH": "enfants"} ,{"ORTH": "s", "OP":"?"}]
patternFamil =[{"TEXT": {"REGEX": "^marié(e)$"}}]
               # {"ORTH": "pacsé(e)", "OP":"?"},{"ORTH": "divorcé(e)", "OP":"?"},{"ORTH": "séparé(e)", "OP":"?"},{"ORTH": "célibataire", "OP":"?"},{"ORTH": "veuf", "OP":"?"} ]
patternSexe = [{"ORTH": "(M)", "OP":"?"},{"ORTH": "(F)", "OP":"?"} ]

In [120]:
matcher = Matcher(nlp.vocab)
matcher.add("mail", add_email, patternMail)
matcher.add("url", add_url, patternUrl)
matcher.add("tel", add_tel, patternTel)
matcher.add("sec", add_ssNum, patternSecS)
matcher.add("date", add_date, patternDate)
matcher.add("Sdate", add_Sdate, patternSDate)
#matcher.add("VerbFem", add_verb, patternVerbFem)
matcher.add("Age", add_age, patternAge)
matcher.add("Age2", add_age, patternAge2)
matcher.add("situation", add_situation_fam, patternFamil)
matcher.add("formation", add_formation, patternFormation)
matcher.add("enfants", add_enfants, patternEnfants)
matcher.add("sexe", add_sexe, patternSexe)
#matcher.add("verb", add_verb, patternVerb)
#matcher.add("adj", add_adj, pattern1)

In [121]:
doc = nlp(merging_inclusive(sentence2))

In [122]:
matches = matcher(doc)

In [123]:
colors = {"URL": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
          "EMAIL": "RGB(241, 13, 105)",
          "TEL" : "RGB(95, 205, 231)",
          "LOC" : "RGB(32, 240, 171)",
          "PER" : "RGB(200, 232, 40)",
          "VERB_FEM" : "linear-gradient(90dgg, #aa4cfc, #fj7ce7)"
         }

In [124]:
options = { "colors": colors}
displacy.render(doc, style="ent", options=options)

## Anonymisation

In [125]:
def anonymize_entities(token):
    if token.ent_iob != 0 and token.ent_type_ == 'PER':
        return '[REDACTEDPER] '
    if token.ent_iob != 0 and token.ent_type_ == 'MISC':
        return '[REDACTEDMISC] '    
    if token.ent_iob != 0 and token.ent_type_ == 'LOC':
        return '[REDACTEDLOC] '
    if token.ent_iob != 0 and token.ent_type_ == 'EMAIL':
        return '[REDACTEDEMAIL] '
    if token.ent_iob != 0 and token.ent_type_ == 'TEL':
        return '[REDACTEDTEL] '
    if token.ent_iob != 0 and token.ent_type_ == 'URL':
        return '[REDACTEDURL] '
    if token.ent_iob != 0 and token.ent_type_ == 'S-DATE':
        return '[REDACTEDS-DATE] '
    if token.ent_iob != 0 and token.ent_type_ == 'DATE':
        return '[REDACTEDDATE] '
    if token.ent_iob != 0 and token.ent_type_ == 'NUM_SEC_SO':
        return '[REDACTEDNUM_SEC_SO] '
    if token.ent_iob != 0 and token.ent_type_ == 'AGE':
        return '[REDACTEDAGE] '
    if token.ent_iob != 0 and token.ent_type_ == 'SIT_FAM':
        return '[REDACTEDSIT_FAM] '
    if token.ent_iob != 0 and token.ent_type_ == 'FORMATION':
        return '[REDACTEDFORMATION] '
    if token.ent_iob != 0 and token.ent_type_ == 'NBR_ENFANTS':
        return '[REDACTEDNBR_ENFANTS] '
    if token.ent_iob != 0 and token.ent_type_ == 'VERB_FEM':
        return token.text[:-1]+' '    
    if token.text == 'elle':
        return 'il '    
    return token.string

def redact_names(nlp_doc):
    for ent in nlp_doc.ents:        
        ent.merge()
    tokens = map(anonymize_entities, nlp_doc)
    return ''.join(tokens)

redact_names(doc)

/Users/amalbedoui/opt/anaconda3/envs/retd/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: [W013] As of v2.1.0, Span.merge is deprecated. Please use the more efficient and less error-prone Doc.retokenize context manager instead.
/Users/amalbedoui/opt/anaconda3/envs/retd/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: [W013] As of v2.1.0, Doc.merge is deprecated. Please use the more efficient and less error-prone Doc.retokenize context manager instead.


"Passionné(e)  par le/la  développement  , PER s'est installé(e)  à [REDACTEDLOC] , [REDACTEDMISC] est [REDACTEDEMAIL] , son  compte  linkedin  est https :[REDACTEDURL] /in/[REDACTEDMISC] et il/il  est joignabl(e)  sur [REDACTEDTEL] . il/il [REDACTEDAGE] vuqu'il est né(e) le/la [REDACTEDDATE] , il/il  est marié(e)  et a 2 enfants  . Son  numéro  de  sécurité sociale [REDACTEDNUM_SEC_SO] . Son  sexe  est [REDACTEDPER] ) . il/il  a  été diplômé(e)  en [REDACTEDDATE] ,  à l'issue d'une  formation  [REDACTEDFORMATION] d'ingénierie  de [REDACTEDS-DATE] à [REDACTEDS-DATE] à l'Ecole ORG à [REDACTEDLOC] . PERParisMISCMISCFORGPalaiseau"